In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
driver = webdriver.Chrome(r"C:/chromedriver_win32/chromedriver.exe")

In [3]:
"""Need to write a dunction that mines all ids for all tournaments availible."""

#Get Basic Tournaments
def getPopularTournaments():
    comp_id = []
    comp_name =[]
    regions =[]
    #Create URL
    driver.get('https://www.whoscored.com')
    #Go to the wanted data table
    base = driver.find_element_by_id('popular')
    elements =base.find_elements_by_tag_name('li')
    #Get Ids
    comp_id.append([re.findall(r'\d+', elements[i].find_element_by_tag_name('a').get_attribute('href'))[1] for i in range(0,len(elements))])
    #Get Regions ID
    regions.append([re.findall(r'\d+', elements[i].find_element_by_tag_name('a').get_attribute('href'))[0] for i in range(0,len(elements))])
    #Get tournament name
    comp_name.append([elements[i].text for i in range(0,len(elements))])
    competitions = pd.DataFrame({'Name': comp_name[0],'ID': comp_id[0],'Region':regions[0]})
    return competitions

tournaments = getPopularTournaments()

In [2]:
# tournaments

In [5]:
# tournaments.to_csv('Tournaments.csv')

In [4]:
#Get Teams Names and ids
def getTeams(tournaments):
    #I will include the europe tournaments here ( they have different page structure )
    tournaments = tournaments[0:15]
    ids = tournaments['ID'].values
    regions = tournaments['Region'].values
    teams = []
    links = []
    team_ids = []
    values=[]
    for i, r in zip(ids, regions):
        #Create URL
        driver.get('https://www.whoscored.com/Regions/'+str(r) + '/Tournaments/' + str(i))
        #Go to the wanted data table
        #base = driver.find_element_by_id('standings-16368-content')
        base = driver.find_element_by_class_name('standings')
        #Get table rows
        rows =base.find_elements_by_tag_name('tr')        
        for item in range(0,len(rows)):
            #Go to specific location to find the team name
            data = rows[item].find_elements_by_tag_name('td')
            teams.append(data[1].text)
            link = rows[item].find_elements_by_class_name('team-link')[0].get_attribute('href')
            team_id = re.search(r'\d+', link).group()
            values.append([i,data[1].text,team_id])
    teams = pd.DataFrame(values,columns=['Tournament_ID','Team_Name','Team_ID'])
    return teams
teams = getTeams(tournaments)

In [5]:
def modifyNumericType(df):
    for column in df.keys():
        df[column] = pd.to_numeric(df[column],errors = 'ignore')
    return df
teams = modifyNumericType(teams)
tournaments = modifyNumericType(tournaments)

In [45]:
"""Get Summary stats Stats Function"""
def getPlayerSummary(relTeams):
    # Create empty list for future data
    values =[]
    players_names = []
    players_ids = []
    for i in relTeams:
        #Get desired url
        driver.get('https://www.whoscored.com/Teams/'+str(i))
        #Wait for the url to load
        driver.implicitly_wait(60)
        base = driver.find_element_by_id('top-player-stats-summary-grid')
        table = base.find_element_by_id('player-table-statistics-body')
        tableRows = table.find_elements_by_tag_name('tr')
        for item in range(0,len(tableRows)):
                #Find the link for each player and extract the id from it
                link = tableRows[item].find_elements_by_class_name('player-link')[0].get_attribute('href')
                players_ids.append(re.search(r'\d+',link).group())
                row     = tableRows[item].find_elements_by_tag_name('td')
                players_names.append(row[2].text)
                #Get Table values
                values.append([row[i].text for i in range(1,len(row))])
    list_of_players = list(zip(players_names, players_ids))
    players = pd.DataFrame(list_of_players, columns = ['Name', 'ID'])
    dfOut = pd.DataFrame(values,columns=["R",'Name', "height", "weight", "apps",'mins','goals','Assists','Yel','Red',
                                                 'SpG','PS%','AerialsWon','MotM','Rating'])
    dfOut = pd.merge(dfOut,players,on='Name')
    return dfOut


def crawlPlayerStats(teams):
    league_ids = teams['Tournament_ID'].unique()
    #optional 
    league_ids = league_ids[9:]
    c = 0        
    for league in league_ids:
        relTeams = teams[teams.Tournament_ID == league]['Team_ID'].values        
        df = getPlayerSummary(relTeams)
        df['League_ID'] = league
        df.to_csv('Player_Stats_%s'%tournaments[tournaments.ID == league]['Name'].values+'.csv')
        c = c+1
    return print('Crawled ' + str(c) +' leagues!')

In [1]:
# crawlPlayerStats(teams)

In [ ]:
# players_summary.to_csv('Players_summary_stats.csv')

In [9]:
"""Get Defensive Stats Function"""

def getDefensiveStats(teams):
    #Create empty list for future data
    values=[]
    #set Url
    driver.get('https://www.whoscored.com/Teams/32/Show/England-Manchester-United')
    #Wait for the page to load.
    driver.implicitly_wait(40)
    #Get the right table by clicking the defensive tab on the data table.
    driver.find_elements_by_id("team-squad-stats-options")[0].find_elements_by_tag_name('li')[1].click()
    base = driver.find_elements_by_id('statistics-table-defensive')
    tableRows = base[0].find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row     = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values)
    #return the data table  
    return df
data = getDefensiveStats(teams)

In [31]:
"""Get Attacking Stats Function"""

def getAttackingStats(teams):
    #Create empty list for future data
    values=[]
    #set Url
    driver.get('https://www.whoscored.com/Teams/32/Show/England-Manchester-United')
    #Wait for the page to load.
    driver.implicitly_wait(40)
    #Get the right table by clicking the defensive tab on the data table.
    driver.find_elements_by_id("team-squad-stats-options")[0].find_elements_by_tag_name('li')[2].click()
    driver.implicitly_wait(40)
    base = driver.find_elements_by_id('statistics-table-offensive')
    tableRows = base[0].find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row     = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values)
    #return the data table  
    return df
data = getAttackingStats(teams)

# Players Match Statistics Crawler

In [21]:
"""Players match Statistics Summary crawler"""
def getPlayerMatchStatisSumm()
values =[]
driver.get('https://www.whoscored.com/Players/97752/MatchStatistics/')
driver.implicitly_wait(40)
base = driver.find_elements_by_id('player-table-statistics-body')
tableRows = base[0].find_elements_by_tag_name('tr')
for item in range(0,len(tableRows)):
    row     = tableRows[item].find_elements_by_tag_name('td')
    values.append([row[i].text for i in range(0,len(row))])
df = pd.DataFrame(values)

In [39]:
"""Match Statistics Defensive Crawler"""
values =[]
driver.get('https://www.whoscored.com/Players/97752/MatchStatistics/')
driver.implicitly_wait(40)
driver.find_elements_by_id("player-matches-stats-options")[0].find_elements_by_tag_name('li')[1].click()
driver.implicitly_wait(40)
base = driver.find_elements_by_id('statistics-table-defensive-matches')
tableRows = base[0].find_elements_by_tag_name('tr')
for item in range(0,len(tableRows)):
    row     = tableRows[item].find_elements_by_tag_name('td')
    values.append([row[i].text for i in range(0,len(row))])
df = pd.DataFrame(values)

In [41]:
"""Match Statistics Ofensive Crawler"""
values =[]
driver.get('https://www.whoscored.com/Players/97752/MatchStatistics/')
driver.implicitly_wait(40)
driver.find_elements_by_id("player-matches-stats-options")[0].find_elements_by_tag_name('li')[2].click()
driver.implicitly_wait(40)
base = driver.find_elements_by_id('statistics-table-offensive-matches')
tableRows = base[0].find_elements_by_tag_name('tr')
for item in range(0,len(tableRows)):
    row     = tableRows[item].find_elements_by_tag_name('td')
    values.append([row[i].text for i in range(0,len(row))])
df = pd.DataFrame(values)

In [ ]:
"""Match Statistics Passing Crawler"""
values =[]
driver.get('https://www.whoscored.com/Players/97752/MatchStatistics/')
driver.implicitly_wait(40)
driver.find_elements_by_id("player-matches-stats-options")[0].find_elements_by_tag_name('li')[3].click()
driver.implicitly_wait(40)
base = driver.find_elements_by_id('statistics-table-passing-matches')
tableRows = base[0].find_elements_by_tag_name('tr')
for item in range(0,len(tableRows)):
    row     = tableRows[item].find_elements_by_tag_name('td')
    values.append([row[i].text for i in range(0,len(row))])
df = pd.DataFrame(values)